### Negation Relations
Since wikidata obviously doesn't support equivalent relations, we have to get them somehow else. For that we just use the aliases of all general properties that we have in the relation_subset. 

In [1]:
import pandas as pd
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from wikidata.client import Client
from googletrans import Translator
from tqdm import tqdm
from collections import defaultdict
import time

In [2]:
from pymongo import MongoClient
client = MongoClient()
wikiclient = Client()

In [3]:
translator = Translator()

In [40]:
negation_relations = pd.read_csv('../data/reasoning/negation/relations_zh_ja.csv')
negation_relations

,id,zh,ja
0,P1034,主要食物來源,主な食料源
1,P457,成立文书,設立文
2,P532,船籍港,船籍港
3,P740,成立地點,結成場所
4,P1817,收件人,宛名
...,...,...,...
509,P31,隶属于,分類
510,P1441,登场作品,登場する作品
511,P828,起因,原因
512,P1542,導致,以下の原因


In [17]:
negation_relations = negation_relations.drop(columns=['Unnamed: 0'])
negation_relations

,id,en,de,es,fr,count,en_negated,de_negated,fr_negated,es_negated
0,P3719,regulated by,reguliert von,regulado por,régi par,469,not regulated by,nicht reguliert von,ne pas régi par,no regulado por
1,P1704,is pollinator of,Bestäuber von,polinizador,pollinisateur de,58,not is pollinator of,nicht Bestäuber von,ne pas pollinisateur de,no polinizador
2,P3173,offers view on,bietet Sicht auf,ofrece vista a,offre une vue sur,875,not offers view on,nicht bietet Sicht auf,ne pas offre une vue sur,no ofrece vista a
3,P3085,qualifies for event,qualifiziert für Turnier,califica para el evento,qualifie pour,2788,not qualifies for event,nicht qualifiziert für Turnier,ne pas qualifie pour,no califica para el evento
4,P4646,representation of,Vorstellung von,representación de,représentation de,165,not representation of,nicht Vorstellung von,ne pas représentation de,no representación de
...,...,...,...,...,...,...,...,...,...,...
789,P31,instance of,ist ein(e),instancia de,nature de l'élément,102954591,not instance of,nicht ist ein(e),ne pas nature de l'élément,no instancia de
790,P1441,present in work,kommt vor in,presente en la obra,présent dans l'œuvre,126014,not present in work,nicht kommt vor in,ne pas présent dans l'œuvre,no presente en la obra
791,P828,has cause,Ursache,causado por,a pour cause,9219,not has cause,nicht Ursache,ne pas a pour cause,no causado por
792,P1542,has effect,Ursache von,causa de,a pour effet,5673,not has effect,nicht Ursache von,ne pas a pour effet,no causa de


In [41]:
no = 'not'
target_languages = ['zh', 'ja']

for target in target_languages:
    # Translate negation
    if target == 'zh':
        no_translate = translator.translate(no, src='en', dest='zh-tw').text
    else:
        no_translate = translator.translate(no, src='en', dest=target).text

    translations = []
    
    for idx, relation in tqdm(negation_relations.iterrows(), total=negation_relations.shape[0]):

        translations.append(no_translate + ' ' + relation[target])

    negation_relations[target + '_negated'] = translations
    
negation_relations

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 514/514 [00:00<00:00, 25335.19it/s]


,id,zh,ja,zh_negated,ja_negated
0,P1034,主要食物來源,主な食料源,不是 主要食物來源,いいえ 主な食料源
1,P457,成立文书,設立文,不是 成立文书,いいえ 設立文
2,P532,船籍港,船籍港,不是 船籍港,いいえ 船籍港
3,P740,成立地點,結成場所,不是 成立地點,いいえ 結成場所
4,P1817,收件人,宛名,不是 收件人,いいえ 宛名
...,...,...,...,...,...
509,P31,隶属于,分類,不是 隶属于,いいえ 分類
510,P1441,登场作品,登場する作品,不是 登场作品,いいえ 登場する作品
511,P828,起因,原因,不是 起因,いいえ 原因
512,P1542,導致,以下の原因,不是 導致,いいえ 以下の原因


In [142]:
# Delete some rows
equivalence_relations = equivalence_relations.drop(equivalence_relations[equivalence_relations['id'] == 'P8929'].index)


In [ ]:
# Rows with None
equivalence_relations[equivalence_relations['en_alias'].isnull()]

In [42]:
# Entity Candidates to Dataframe
df = pd.DataFrame(negation_relations)
df

,id,zh,ja,zh_negated,ja_negated
0,P1034,主要食物來源,主な食料源,不是 主要食物來源,いいえ 主な食料源
1,P457,成立文书,設立文,不是 成立文书,いいえ 設立文
2,P532,船籍港,船籍港,不是 船籍港,いいえ 船籍港
3,P740,成立地點,結成場所,不是 成立地點,いいえ 結成場所
4,P1817,收件人,宛名,不是 收件人,いいえ 宛名
...,...,...,...,...,...
509,P31,隶属于,分類,不是 隶属于,いいえ 分類
510,P1441,登场作品,登場する作品,不是 登场作品,いいえ 登場する作品
511,P828,起因,原因,不是 起因,いいえ 原因
512,P1542,導致,以下の原因,不是 導致,いいえ 以下の原因


In [43]:
# Save as CSV for manual cleanup
df.to_csv('../data/reasoning/negation_general_zh_ja.csv', index=False)